In [9]:
import pandas as pd

games = pd.read_csv('game_data_start.csv')
games.head()

,Game ID,Date,Team,Opp,Result,Pts,Opp_Pts,ORtg
0,0,5/16/21,ATL,HOU,W 124-95,124,95,121.4
1,1,5/16/21,HOU,ATL,L 95-124,95,124,93.0
2,2,5/16/21,BKN,CLE,W 123-109,123,109,125.1
3,3,5/16/21,CLE,BKN,L 109-123,109,123,110.9
4,4,5/16/21,CHI,MIL,W 118-112,118,112,116.1


In [5]:
#Check that we have 12 games for every team.
games['Team'].value_counts()

GSW    12
CHI    12
NOP    12
ATL    12
TOR    12
NYK    12
LAL    12
OKC    12
LAC    12
SAC    12
POR    12
HOU    12
BKN    12
DAL    12
SAS    12
WAS    12
BOS    12
MIA    12
IND    12
UTA    12
PHX    12
MIN    12
MEM    12
CLE    12
DEN    12
ORL    12
PHI    12
MIL    12
CHA    12
DET    12
Name: Team, dtype: int64

In [6]:
#Create point differential column.
games['Pt_Diff'] = games['Pts'] - games['Opp_Pts']
games.head()

,Game ID,Date,Team,Opp,Result,Pts,Opp_Pts,ORtg,Pt_Diff
0,0,5/16/21,ATL,HOU,W 124-95,124,95,121.4,29
1,1,5/16/21,HOU,ATL,L 95-124,95,124,93.0,-29
2,2,5/16/21,BKN,CLE,W 123-109,123,109,125.1,14
3,3,5/16/21,CLE,BKN,L 109-123,109,123,110.9,-14
4,4,5/16/21,CHI,MIL,W 118-112,118,112,116.1,6


In [7]:
#Use point differential to create win/loss column.
#Now we have final set we want to work with.
W_L_list = []
for elem in games['Pt_Diff']:
    if elem > 0:
        W_L_list.append(1)
    elif elem < 0:
        W_L_list.append(0)
        
games['W/L'] = W_L_list
games.head()

,Game ID,Date,Team,Opp,Result,Pts,Opp_Pts,ORtg,Pt_Diff,W/L
0,0,5/16/21,ATL,HOU,W 124-95,124,95,121.4,29,1
1,1,5/16/21,HOU,ATL,L 95-124,95,124,93.0,-29,0
2,2,5/16/21,BKN,CLE,W 123-109,123,109,125.1,14,1
3,3,5/16/21,CLE,BKN,L 109-123,109,123,110.9,-14,0
4,4,5/16/21,CHI,MIL,W 118-112,118,112,116.1,6,1


In [8]:
#Map team abbreviation to team id for the NBA stats website.
team_to_id = {
    'ATL':'1610612737',
    'BOS':'1610612738',
    'CLE':'1610612739',
    'NOP':'1610612740',
    'CHI':'1610612741',
    'DAL':'1610612742',
    'DEN':'1610612743',
    'GSW':'1610612744',
    'HOU':'1610612745',
    'LAC':'1610612746',
    'LAL':'1610612747',
    'MIA':'1610612748',
    'MIL':'1610612749',
    'MIN':'1610612750',
    'BKN':'1610612751',
    'NYK':'1610612752',
    'ORL':'1610612753',
    'IND':'1610612754',
    'PHI':'1610612755',
    'PHX':'1610612756',
    'POR':'1610612757',
    'SAC':'1610612758',
    'SAS':'1610612759',
    'OKC':'1610612760',
    'TOR':'1610612761',
    'UTA':'1610612762',
    'MEM':'1610612763',
    'WAS':'1610612764',
    'DET':'1610612765',
    'CHA':'1610612766'   
}

In [7]:
#Might have to scrape data in chunks, rather than everything at the same time.
#Games 0-29
from nba_api.stats.endpoints import leaguedashplayershotlocations
import nba_api
import json
import pandas as pd

pd.set_option('display.max_columns', None)

master_dataset_list = []

for i in range(30):
    game = games.iloc[i]
    game_id = game['Game ID']
    date = game['Date']
    team = game['Team']
    team_id = team_to_id[team]
    opp = game['Opp']
    result = game['Result']
    pts = game['Pts']
    opp_pts= game['Opp_Pts']
    pt_diff = game['Pt_Diff']
    ortg = game['ORtg']
    win = game['W/L']
    game_list = [game_id, date, team, team_id, opp, result, pts, opp_pts, pt_diff, ortg, win]


    raw_data = leaguedashplayershotlocations.LeagueDashPlayerShotLocations(
                date_from_nullable = date,
                date_to_nullable = date, 
                team_id_nullable = team_id
            )
    content = json.loads(raw_data.get_json()) 
    
    resultSet = content['resultSets']
    data = resultSet['rowSet']

    for player in data:
        
        #Restricted Area
        ra = game_list + player[:6] + ['Restricted Area']
        for i in range(6,9):
            ra.append(player[i])
        
        #In the Paint
        paint = game_list + player[:6] + ['In The Paint (Non-RA)']
        for i in range(9,12):
            paint.append(player[i])
        
        #Mid-Range
        mid = game_list + player[:6] + ['Mid-Range']
        for i in range(12,15):
            mid.append(player[i])
        
        #Left Corner 3
        lc3 = game_list + player[:6] + ['Left Corner 3']
        for i in range(15,18):
            lc3.append(player[i])
        
        #Right Corner 3
        rc3 = game_list + player[:6] + ['Right Corner 3']
        for i in range(18,21):
            rc3.append(player[i])
        
        #Above the Break 3
        above3 = game_list + player[:6] + ['Above the Break 3']
        for i in range(21,24):
            above3.append(player[i])
        
        #Backcourt
        back = game_list + player[:6] + ['Backcourt']
        for i in range(24,27):
            back.append(player[i])
        
        #Corner 3
        corner = game_list + player[:6] + ['Corner 3']
        for i in range(27,30):
            corner.append(player[i])
            
        master_dataset_list.append(ra)
        master_dataset_list.append(paint)
        master_dataset_list.append(mid)
        master_dataset_list.append(lc3)
        master_dataset_list.append(rc3)
        master_dataset_list.append(above3)
        master_dataset_list.append(back)
        master_dataset_list.append(corner)

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [ ]:
len(master_dataset_list)

In [ ]:
#Games 30-59
for i in range(30,60):
    game = games.iloc[i]
    game_id = game['Game ID']
    date = game['Date']
    team = game['Team']
    team_id = team_to_id[team]
    opp = game['Opp']
    result = game['Result']
    pts = game['Pts']
    opp_pts= game['Opp_Pts']
    pt_diff = game['Pt_Diff']
    ortg = game['ORtg']
    win = game['W/L']
    game_list = [game_id, date, team, team_id, opp, result, pts, opp_pts, pt_diff, ortg, win]


    raw_data = leaguedashplayershotlocations.LeagueDashPlayerShotLocations(
                date_from_nullable = date,
                date_to_nullable = date, 
                team_id_nullable = team_id
            )
    content = json.loads(raw_data.get_json()) 
    
    resultSet = content['resultSets']
    data = resultSet['rowSet']

    for player in data:
        
        #Restricted Area
        ra = game_list + player[:6] + ['Restricted Area']
        for i in range(6,9):
            ra.append(player[i])
        
        #In the Paint
        paint = game_list + player[:6] + ['In The Paint (Non-RA)']
        for i in range(9,12):
            paint.append(player[i])
        
        #Mid-Range
        mid = game_list + player[:6] + ['Mid-Range']
        for i in range(12,15):
            mid.append(player[i])
        
        #Left Corner 3
        lc3 = game_list + player[:6] + ['Left Corner 3']
        for i in range(15,18):
            lc3.append(player[i])
        
        #Right Corner 3
        rc3 = game_list + player[:6] + ['Right Corner 3']
        for i in range(18,21):
            rc3.append(player[i])
        
        #Above the Break 3
        above3 = game_list + player[:6] + ['Above the Break 3']
        for i in range(21,24):
            above3.append(player[i])
        
        #Backcourt
        back = game_list + player[:6] + ['Backcourt']
        for i in range(24,27):
            back.append(player[i])
        
        #Corner 3
        corner = game_list + player[:6] + ['Corner 3']
        for i in range(27,30):
            corner.append(player[i])
            
        master_dataset_list.append(ra)
        master_dataset_list.append(paint)
        master_dataset_list.append(mid)
        master_dataset_list.append(lc3)
        master_dataset_list.append(rc3)
        master_dataset_list.append(above3)
        master_dataset_list.append(back)
        master_dataset_list.append(corner)

In [ ]:
len(master_dataset_list)

In [ ]:
#Games 60-89
for i in range(60,90):
    game = games.iloc[i]
    game_id = game['Game ID']
    date = game['Date']
    team = game['Team']
    team_id = team_to_id[team]
    opp = game['Opp']
    result = game['Result']
    pts = game['Pts']
    opp_pts= game['Opp_Pts']
    pt_diff = game['Pt_Diff']
    ortg = game['ORtg']
    win = game['W/L']
    game_list = [game_id, date, team, team_id, opp, result, pts, opp_pts, pt_diff, ortg, win]


    raw_data = leaguedashplayershotlocations.LeagueDashPlayerShotLocations(
                date_from_nullable = date,
                date_to_nullable = date, 
                team_id_nullable = team_id
            )
    content = json.loads(raw_data.get_json()) 
    
    resultSet = content['resultSets']
    data = resultSet['rowSet']

    for player in data:
        
        #Restricted Area
        ra = game_list + player[:6] + ['Restricted Area']
        for i in range(6,9):
            ra.append(player[i])
        
        #In the Paint
        paint = game_list + player[:6] + ['In The Paint (Non-RA)']
        for i in range(9,12):
            paint.append(player[i])
        
        #Mid-Range
        mid = game_list + player[:6] + ['Mid-Range']
        for i in range(12,15):
            mid.append(player[i])
        
        #Left Corner 3
        lc3 = game_list + player[:6] + ['Left Corner 3']
        for i in range(15,18):
            lc3.append(player[i])
        
        #Right Corner 3
        rc3 = game_list + player[:6] + ['Right Corner 3']
        for i in range(18,21):
            rc3.append(player[i])
        
        #Above the Break 3
        above3 = game_list + player[:6] + ['Above the Break 3']
        for i in range(21,24):
            above3.append(player[i])
        
        #Backcourt
        back = game_list + player[:6] + ['Backcourt']
        for i in range(24,27):
            back.append(player[i])
        
        #Corner 3
        corner = game_list + player[:6] + ['Corner 3']
        for i in range(27,30):
            corner.append(player[i])
            
        master_dataset_list.append(ra)
        master_dataset_list.append(paint)
        master_dataset_list.append(mid)
        master_dataset_list.append(lc3)
        master_dataset_list.append(rc3)
        master_dataset_list.append(above3)
        master_dataset_list.append(back)
        master_dataset_list.append(corner)

In [ ]:
len(master_dataset_list)

In [ ]:
#Games 90-119
for i in range(90,120):
    game = games.iloc[i]
    game_id = game['Game ID']
    date = game['Date']
    team = game['Team']
    team_id = team_to_id[team]
    opp = game['Opp']
    result = game['Result']
    pts = game['Pts']
    opp_pts= game['Opp_Pts']
    pt_diff = game['Pt_Diff']
    ortg = game['ORtg']
    win = game['W/L']
    game_list = [game_id, date, team, team_id, opp, result, pts, opp_pts, pt_diff, ortg, win]


    raw_data = leaguedashplayershotlocations.LeagueDashPlayerShotLocations(
                date_from_nullable = date,
                date_to_nullable = date, 
                team_id_nullable = team_id
            )
    content = json.loads(raw_data.get_json()) 
    
    resultSet = content['resultSets']
    data = resultSet['rowSet']

    for player in data:
        
        #Restricted Area
        ra = game_list + player[:6] + ['Restricted Area']
        for i in range(6,9):
            ra.append(player[i])
        
        #In the Paint
        paint = game_list + player[:6] + ['In The Paint (Non-RA)']
        for i in range(9,12):
            paint.append(player[i])
        
        #Mid-Range
        mid = game_list + player[:6] + ['Mid-Range']
        for i in range(12,15):
            mid.append(player[i])
        
        #Left Corner 3
        lc3 = game_list + player[:6] + ['Left Corner 3']
        for i in range(15,18):
            lc3.append(player[i])
        
        #Right Corner 3
        rc3 = game_list + player[:6] + ['Right Corner 3']
        for i in range(18,21):
            rc3.append(player[i])
        
        #Above the Break 3
        above3 = game_list + player[:6] + ['Above the Break 3']
        for i in range(21,24):
            above3.append(player[i])
        
        #Backcourt
        back = game_list + player[:6] + ['Backcourt']
        for i in range(24,27):
            back.append(player[i])
        
        #Corner 3
        corner = game_list + player[:6] + ['Corner 3']
        for i in range(27,30):
            corner.append(player[i])
            
        master_dataset_list.append(ra)
        master_dataset_list.append(paint)
        master_dataset_list.append(mid)
        master_dataset_list.append(lc3)
        master_dataset_list.append(rc3)
        master_dataset_list.append(above3)
        master_dataset_list.append(back)
        master_dataset_list.append(corner)

In [ ]:
len(master_dataset_list)

In [ ]:
#Games 120-149
for i in range(120,150):
    game = games.iloc[i]
    game_id = game['Game ID']
    date = game['Date']
    team = game['Team']
    team_id = team_to_id[team]
    opp = game['Opp']
    result = game['Result']
    pts = game['Pts']
    opp_pts= game['Opp_Pts']
    pt_diff = game['Pt_Diff']
    ortg = game['ORtg']
    win = game['W/L']
    game_list = [game_id, date, team, team_id, opp, result, pts, opp_pts, pt_diff, ortg, win]


    raw_data = leaguedashplayershotlocations.LeagueDashPlayerShotLocations(
                date_from_nullable = date,
                date_to_nullable = date, 
                team_id_nullable = team_id
            )
    content = json.loads(raw_data.get_json()) 
    
    resultSet = content['resultSets']
    data = resultSet['rowSet']

    for player in data:
        
        #Restricted Area
        ra = game_list + player[:6] + ['Restricted Area']
        for i in range(6,9):
            ra.append(player[i])
        
        #In the Paint
        paint = game_list + player[:6] + ['In The Paint (Non-RA)']
        for i in range(9,12):
            paint.append(player[i])
        
        #Mid-Range
        mid = game_list + player[:6] + ['Mid-Range']
        for i in range(12,15):
            mid.append(player[i])
        
        #Left Corner 3
        lc3 = game_list + player[:6] + ['Left Corner 3']
        for i in range(15,18):
            lc3.append(player[i])
        
        #Right Corner 3
        rc3 = game_list + player[:6] + ['Right Corner 3']
        for i in range(18,21):
            rc3.append(player[i])
        
        #Above the Break 3
        above3 = game_list + player[:6] + ['Above the Break 3']
        for i in range(21,24):
            above3.append(player[i])
        
        #Backcourt
        back = game_list + player[:6] + ['Backcourt']
        for i in range(24,27):
            back.append(player[i])
        
        #Corner 3
        corner = game_list + player[:6] + ['Corner 3']
        for i in range(27,30):
            corner.append(player[i])
            
        master_dataset_list.append(ra)
        master_dataset_list.append(paint)
        master_dataset_list.append(mid)
        master_dataset_list.append(lc3)
        master_dataset_list.append(rc3)
        master_dataset_list.append(above3)
        master_dataset_list.append(back)
        master_dataset_list.append(corner)

In [ ]:
len(master_dataset_list)

In [ ]:
#Games 150-179
for i in range(150,180):
    game = games.iloc[i]
    game_id = game['Game ID']
    date = game['Date']
    team = game['Team']
    team_id = team_to_id[team]
    opp = game['Opp']
    result = game['Result']
    pts = game['Pts']
    opp_pts= game['Opp_Pts']
    pt_diff = game['Pt_Diff']
    ortg = game['ORtg']
    win = game['W/L']
    game_list = [game_id, date, team, team_id, opp, result, pts, opp_pts, pt_diff, ortg, win]


    raw_data = leaguedashplayershotlocations.LeagueDashPlayerShotLocations(
                date_from_nullable = date,
                date_to_nullable = date, 
                team_id_nullable = team_id
            )
    content = json.loads(raw_data.get_json()) 
    
    resultSet = content['resultSets']
    data = resultSet['rowSet']

    for player in data:
        
        #Restricted Area
        ra = game_list + player[:6] + ['Restricted Area']
        for i in range(6,9):
            ra.append(player[i])
        
        #In the Paint
        paint = game_list + player[:6] + ['In The Paint (Non-RA)']
        for i in range(9,12):
            paint.append(player[i])
        
        #Mid-Range
        mid = game_list + player[:6] + ['Mid-Range']
        for i in range(12,15):
            mid.append(player[i])
        
        #Left Corner 3
        lc3 = game_list + player[:6] + ['Left Corner 3']
        for i in range(15,18):
            lc3.append(player[i])
        
        #Right Corner 3
        rc3 = game_list + player[:6] + ['Right Corner 3']
        for i in range(18,21):
            rc3.append(player[i])
        
        #Above the Break 3
        above3 = game_list + player[:6] + ['Above the Break 3']
        for i in range(21,24):
            above3.append(player[i])
        
        #Backcourt
        back = game_list + player[:6] + ['Backcourt']
        for i in range(24,27):
            back.append(player[i])
        
        #Corner 3
        corner = game_list + player[:6] + ['Corner 3']
        for i in range(27,30):
            corner.append(player[i])
            
        master_dataset_list.append(ra)
        master_dataset_list.append(paint)
        master_dataset_list.append(mid)
        master_dataset_list.append(lc3)
        master_dataset_list.append(rc3)
        master_dataset_list.append(above3)
        master_dataset_list.append(back)
        master_dataset_list.append(corner)

In [ ]:
len(master_dataset_list)

In [ ]:
#Games 180-209
for i in range(180,210):
    game = games.iloc[i]
    game_id = game['Game ID']
    date = game['Date']
    team = game['Team']
    team_id = team_to_id[team]
    opp = game['Opp']
    result = game['Result']
    pts = game['Pts']
    opp_pts= game['Opp_Pts']
    pt_diff = game['Pt_Diff']
    ortg = game['ORtg']
    win = game['W/L']
    game_list = [game_id, date, team, team_id, opp, result, pts, opp_pts, pt_diff, ortg, win]


    raw_data = leaguedashplayershotlocations.LeagueDashPlayerShotLocations(
                date_from_nullable = date,
                date_to_nullable = date, 
                team_id_nullable = team_id
            )
    content = json.loads(raw_data.get_json()) 
    
    resultSet = content['resultSets']
    data = resultSet['rowSet']

    for player in data:
        
        #Restricted Area
        ra = game_list + player[:6] + ['Restricted Area']
        for i in range(6,9):
            ra.append(player[i])
        
        #In the Paint
        paint = game_list + player[:6] + ['In The Paint (Non-RA)']
        for i in range(9,12):
            paint.append(player[i])
        
        #Mid-Range
        mid = game_list + player[:6] + ['Mid-Range']
        for i in range(12,15):
            mid.append(player[i])
        
        #Left Corner 3
        lc3 = game_list + player[:6] + ['Left Corner 3']
        for i in range(15,18):
            lc3.append(player[i])
        
        #Right Corner 3
        rc3 = game_list + player[:6] + ['Right Corner 3']
        for i in range(18,21):
            rc3.append(player[i])
        
        #Above the Break 3
        above3 = game_list + player[:6] + ['Above the Break 3']
        for i in range(21,24):
            above3.append(player[i])
        
        #Backcourt
        back = game_list + player[:6] + ['Backcourt']
        for i in range(24,27):
            back.append(player[i])
        
        #Corner 3
        corner = game_list + player[:6] + ['Corner 3']
        for i in range(27,30):
            corner.append(player[i])
            
        master_dataset_list.append(ra)
        master_dataset_list.append(paint)
        master_dataset_list.append(mid)
        master_dataset_list.append(lc3)
        master_dataset_list.append(rc3)
        master_dataset_list.append(above3)
        master_dataset_list.append(back)
        master_dataset_list.append(corner)

In [ ]:
len(master_dataset_list)

In [ ]:
#Games 210-239
for i in range(210,240):
    game = games.iloc[i]
    game_id = game['Game ID']
    date = game['Date']
    team = game['Team']
    team_id = team_to_id[team]
    opp = game['Opp']
    result = game['Result']
    pts = game['Pts']
    opp_pts= game['Opp_Pts']
    pt_diff = game['Pt_Diff']
    ortg = game['ORtg']
    win = game['W/L']
    game_list = [game_id, date, team, team_id, opp, result, pts, opp_pts, pt_diff, ortg, win]


    raw_data = leaguedashplayershotlocations.LeagueDashPlayerShotLocations(
                date_from_nullable = date,
                date_to_nullable = date, 
                team_id_nullable = team_id
            )
    content = json.loads(raw_data.get_json()) 
    
    resultSet = content['resultSets']
    data = resultSet['rowSet']

    for player in data:
        
        #Restricted Area
        ra = game_list + player[:6] + ['Restricted Area']
        for i in range(6,9):
            ra.append(player[i])
        
        #In the Paint
        paint = game_list + player[:6] + ['In The Paint (Non-RA)']
        for i in range(9,12):
            paint.append(player[i])
        
        #Mid-Range
        mid = game_list + player[:6] + ['Mid-Range']
        for i in range(12,15):
            mid.append(player[i])
        
        #Left Corner 3
        lc3 = game_list + player[:6] + ['Left Corner 3']
        for i in range(15,18):
            lc3.append(player[i])
        
        #Right Corner 3
        rc3 = game_list + player[:6] + ['Right Corner 3']
        for i in range(18,21):
            rc3.append(player[i])
        
        #Above the Break 3
        above3 = game_list + player[:6] + ['Above the Break 3']
        for i in range(21,24):
            above3.append(player[i])
        
        #Backcourt
        back = game_list + player[:6] + ['Backcourt']
        for i in range(24,27):
            back.append(player[i])
        
        #Corner 3
        corner = game_list + player[:6] + ['Corner 3']
        for i in range(27,30):
            corner.append(player[i])
            
        master_dataset_list.append(ra)
        master_dataset_list.append(paint)
        master_dataset_list.append(mid)
        master_dataset_list.append(lc3)
        master_dataset_list.append(rc3)
        master_dataset_list.append(above3)
        master_dataset_list.append(back)
        master_dataset_list.append(corner)

In [ ]:
len(master_dataset_list)

In [ ]:
#Games 240-269
for i in range(240,270):
    game = games.iloc[i]
    game_id = game['Game ID']
    date = game['Date']
    team = game['Team']
    team_id = team_to_id[team]
    opp = game['Opp']
    result = game['Result']
    pts = game['Pts']
    opp_pts= game['Opp_Pts']
    pt_diff = game['Pt_Diff']
    ortg = game['ORtg']
    win = game['W/L']
    game_list = [game_id, date, team, team_id, opp, result, pts, opp_pts, pt_diff, ortg, win]


    raw_data = leaguedashplayershotlocations.LeagueDashPlayerShotLocations(
                date_from_nullable = date,
                date_to_nullable = date, 
                team_id_nullable = team_id
            )
    content = json.loads(raw_data.get_json()) 
    
    resultSet = content['resultSets']
    data = resultSet['rowSet']

    for player in data:
        
        #Restricted Area
        ra = game_list + player[:6] + ['Restricted Area']
        for i in range(6,9):
            ra.append(player[i])
        
        #In the Paint
        paint = game_list + player[:6] + ['In The Paint (Non-RA)']
        for i in range(9,12):
            paint.append(player[i])
        
        #Mid-Range
        mid = game_list + player[:6] + ['Mid-Range']
        for i in range(12,15):
            mid.append(player[i])
        
        #Left Corner 3
        lc3 = game_list + player[:6] + ['Left Corner 3']
        for i in range(15,18):
            lc3.append(player[i])
        
        #Right Corner 3
        rc3 = game_list + player[:6] + ['Right Corner 3']
        for i in range(18,21):
            rc3.append(player[i])
        
        #Above the Break 3
        above3 = game_list + player[:6] + ['Above the Break 3']
        for i in range(21,24):
            above3.append(player[i])
        
        #Backcourt
        back = game_list + player[:6] + ['Backcourt']
        for i in range(24,27):
            back.append(player[i])
        
        #Corner 3
        corner = game_list + player[:6] + ['Corner 3']
        for i in range(27,30):
            corner.append(player[i])
            
        master_dataset_list.append(ra)
        master_dataset_list.append(paint)
        master_dataset_list.append(mid)
        master_dataset_list.append(lc3)
        master_dataset_list.append(rc3)
        master_dataset_list.append(above3)
        master_dataset_list.append(back)
        master_dataset_list.append(corner)

In [ ]:
len(master_dataset_list)

In [ ]:
#Games 270-299
for i in range(270,300):
    game = games.iloc[i]
    game_id = game['Game ID']
    date = game['Date']
    team = game['Team']
    team_id = team_to_id[team]
    opp = game['Opp']
    result = game['Result']
    pts = game['Pts']
    opp_pts= game['Opp_Pts']
    pt_diff = game['Pt_Diff']
    ortg = game['ORtg']
    win = game['W/L']
    game_list = [game_id, date, team, team_id, opp, result, pts, opp_pts, pt_diff, ortg, win]


    raw_data = leaguedashplayershotlocations.LeagueDashPlayerShotLocations(
                date_from_nullable = date,
                date_to_nullable = date, 
                team_id_nullable = team_id
            )
    content = json.loads(raw_data.get_json()) 
    
    resultSet = content['resultSets']
    data = resultSet['rowSet']

    for player in data:
        
        #Restricted Area
        ra = game_list + player[:6] + ['Restricted Area']
        for i in range(6,9):
            ra.append(player[i])
        
        #In the Paint
        paint = game_list + player[:6] + ['In The Paint (Non-RA)']
        for i in range(9,12):
            paint.append(player[i])
        
        #Mid-Range
        mid = game_list + player[:6] + ['Mid-Range']
        for i in range(12,15):
            mid.append(player[i])
        
        #Left Corner 3
        lc3 = game_list + player[:6] + ['Left Corner 3']
        for i in range(15,18):
            lc3.append(player[i])
        
        #Right Corner 3
        rc3 = game_list + player[:6] + ['Right Corner 3']
        for i in range(18,21):
            rc3.append(player[i])
        
        #Above the Break 3
        above3 = game_list + player[:6] + ['Above the Break 3']
        for i in range(21,24):
            above3.append(player[i])
        
        #Backcourt
        back = game_list + player[:6] + ['Backcourt']
        for i in range(24,27):
            back.append(player[i])
        
        #Corner 3
        corner = game_list + player[:6] + ['Corner 3']
        for i in range(27,30):
            corner.append(player[i])
            
        master_dataset_list.append(ra)
        master_dataset_list.append(paint)
        master_dataset_list.append(mid)
        master_dataset_list.append(lc3)
        master_dataset_list.append(rc3)
        master_dataset_list.append(above3)
        master_dataset_list.append(back)
        master_dataset_list.append(corner)

In [ ]:
len(master_dataset_list)

In [ ]:
#Games 300-329
for i in range(300,330):
    game = games.iloc[i]
    game_id = game['Game ID']
    date = game['Date']
    team = game['Team']
    team_id = team_to_id[team]
    opp = game['Opp']
    result = game['Result']
    pts = game['Pts']
    opp_pts= game['Opp_Pts']
    pt_diff = game['Pt_Diff']
    ortg = game['ORtg']
    win = game['W/L']
    game_list = [game_id, date, team, team_id, opp, result, pts, opp_pts, pt_diff, ortg, win]


    raw_data = leaguedashplayershotlocations.LeagueDashPlayerShotLocations(
                date_from_nullable = date,
                date_to_nullable = date, 
                team_id_nullable = team_id
            )
    content = json.loads(raw_data.get_json()) 
    
    resultSet = content['resultSets']
    data = resultSet['rowSet']

    for player in data:
        
        #Restricted Area
        ra = game_list + player[:6] + ['Restricted Area']
        for i in range(6,9):
            ra.append(player[i])
        
        #In the Paint
        paint = game_list + player[:6] + ['In The Paint (Non-RA)']
        for i in range(9,12):
            paint.append(player[i])
        
        #Mid-Range
        mid = game_list + player[:6] + ['Mid-Range']
        for i in range(12,15):
            mid.append(player[i])
        
        #Left Corner 3
        lc3 = game_list + player[:6] + ['Left Corner 3']
        for i in range(15,18):
            lc3.append(player[i])
        
        #Right Corner 3
        rc3 = game_list + player[:6] + ['Right Corner 3']
        for i in range(18,21):
            rc3.append(player[i])
        
        #Above the Break 3
        above3 = game_list + player[:6] + ['Above the Break 3']
        for i in range(21,24):
            above3.append(player[i])
        
        #Backcourt
        back = game_list + player[:6] + ['Backcourt']
        for i in range(24,27):
            back.append(player[i])
        
        #Corner 3
        corner = game_list + player[:6] + ['Corner 3']
        for i in range(27,30):
            corner.append(player[i])
            
        master_dataset_list.append(ra)
        master_dataset_list.append(paint)
        master_dataset_list.append(mid)
        master_dataset_list.append(lc3)
        master_dataset_list.append(rc3)
        master_dataset_list.append(above3)
        master_dataset_list.append(back)
        master_dataset_list.append(corner)

In [ ]:
len(master_dataset_list)

In [ ]:
#Games 330-359
for i in range(330,360):
    game = games.iloc[i]
    game_id = game['Game ID']
    date = game['Date']
    team = game['Team']
    team_id = team_to_id[team]
    opp = game['Opp']
    result = game['Result']
    pts = game['Pts']
    opp_pts= game['Opp_Pts']
    pt_diff = game['Pt_Diff']
    ortg = game['ORtg']
    win = game['W/L']
    game_list = [game_id, date, team, team_id, opp, result, pts, opp_pts, pt_diff, ortg, win]


    raw_data = leaguedashplayershotlocations.LeagueDashPlayerShotLocations(
                date_from_nullable = date,
                date_to_nullable = date, 
                team_id_nullable = team_id
            )
    content = json.loads(raw_data.get_json()) 
    
    resultSet = content['resultSets']
    data = resultSet['rowSet']

    for player in data:
        
        #Restricted Area
        ra = game_list + player[:6] + ['Restricted Area']
        for i in range(6,9):
            ra.append(player[i])
        
        #In the Paint
        paint = game_list + player[:6] + ['In The Paint (Non-RA)']
        for i in range(9,12):
            paint.append(player[i])
        
        #Mid-Range
        mid = game_list + player[:6] + ['Mid-Range']
        for i in range(12,15):
            mid.append(player[i])
        
        #Left Corner 3
        lc3 = game_list + player[:6] + ['Left Corner 3']
        for i in range(15,18):
            lc3.append(player[i])
        
        #Right Corner 3
        rc3 = game_list + player[:6] + ['Right Corner 3']
        for i in range(18,21):
            rc3.append(player[i])
        
        #Above the Break 3
        above3 = game_list + player[:6] + ['Above the Break 3']
        for i in range(21,24):
            above3.append(player[i])
        
        #Backcourt
        back = game_list + player[:6] + ['Backcourt']
        for i in range(24,27):
            back.append(player[i])
        
        #Corner 3
        corner = game_list + player[:6] + ['Corner 3']
        for i in range(27,30):
            corner.append(player[i])
            
        master_dataset_list.append(ra)
        master_dataset_list.append(paint)
        master_dataset_list.append(mid)
        master_dataset_list.append(lc3)
        master_dataset_list.append(rc3)
        master_dataset_list.append(above3)
        master_dataset_list.append(back)
        master_dataset_list.append(corner)
        
#game id, date, team, team id, opp, result, pts, opp pts, pt diff, ortg, win,
#player id, player name, nickname, team id, team, age, zone, fgm, fga, fg%       
        
master_dataset_cols = ['game_id',
                       'date',
                       'team',
                       'team_id',
                       'opp',
                       'result',
                       'pts',
                       'opp_pts',
                       'pt_diff',
                       'ortg',
                       'win',
                       'player_id',
                       'player_name',
                       'nickname',
                       'team_id',
                       'team_abbreviation',
                       'age',
                       'zone',
                       'fgm',
                       'fga',
                       'fg%'
                      ]

master_dataset = pd.DataFrame(master_dataset_list, columns = master_dataset_cols)
master_dataset.head()

In [ ]:
len(master_dataset_list)

In [10]:
range(30)
range(30,60)
range(60,90)
range(90,120)
range(120,150)
range(150,180)
range(180,210)
range(210,240)
range(240,270)
range(270,300)
range(300,330)
range(330,360)

range(330, 360)

In [ ]:
#All games at once.
for i in range(len(games)):
    game = games.iloc[i]
    game_id = game['Game ID']
    date = game['Date']
    team = game['Team']
    team_id = team_to_id[team]
    opp = game['Opp']
    result = game['Result']
    pts = game['Pts']
    opp_pts= game['Opp_Pts']
    pt_diff = game['Pt_Diff']
    ortg = game['ORtg']
    win = game['W/L']
    game_list = [game_id, date, team, team_id, opp, result, pts, opp_pts, pt_diff, ortg, win]


    raw_data = leaguedashplayershotlocations.LeagueDashPlayerShotLocations(
                date_from_nullable = date,
                date_to_nullable = date, 
                team_id_nullable = team_id
            )
    content = json.loads(raw_data.get_json()) 
    
    resultSet = content['resultSets']
    data = resultSet['rowSet']

    for player in data:
        
        #Restricted Area
        ra = game_list + player[:6] + ['Restricted Area']
        for i in range(6,9):
            ra.append(player[i])
        
        #In the Paint
        paint = game_list + player[:6] + ['In The Paint (Non-RA)']
        for i in range(9,12):
            paint.append(player[i])
        
        #Mid-Range
        mid = game_list + player[:6] + ['Mid-Range']
        for i in range(12,15):
            mid.append(player[i])
        
        #Left Corner 3
        lc3 = game_list + player[:6] + ['Left Corner 3']
        for i in range(15,18):
            lc3.append(player[i])
        
        #Right Corner 3
        rc3 = game_list + player[:6] + ['Right Corner 3']
        for i in range(18,21):
            rc3.append(player[i])
        
        #Above the Break 3
        above3 = game_list + player[:6] + ['Above the Break 3']
        for i in range(21,24):
            above3.append(player[i])
        
        #Backcourt
        back = game_list + player[:6] + ['Backcourt']
        for i in range(24,27):
            back.append(player[i])
        
        #Corner 3
        corner = game_list + player[:6] + ['Corner 3']
        for i in range(27,30):
            corner.append(player[i])
            
        master_dataset_list.append(ra)
        master_dataset_list.append(paint)
        master_dataset_list.append(mid)
        master_dataset_list.append(lc3)
        master_dataset_list.append(rc3)
        master_dataset_list.append(above3)
        master_dataset_list.append(back)
        master_dataset_list.append(corner)
        
#game id, date, team, team id, opp, result, pts, opp pts, pt diff, ortg, win,
#player id, player name, nickname, team id, team, age, zone, fgm, fga, fg%       
        
master_dataset_cols = ['game_id',
                       'date',
                       'team',
                       'team_id',
                       'opp',
                       'result',
                       'pts',
                       'opp_pts',
                       'pt_diff',
                       'ortg',
                       'win',
                       'player_id',
                       'player_name',
                       'nickname',
                       'team_id',
                       'team_abbreviation',
                       'age',
                       'zone',
                       'fgm',
                       'fga',
                       'fg%'
                      ]

master_dataset = pd.DataFrame(master_dataset_list, columns = master_dataset_cols)
master_dataset.head()